In [1]:
using PorousMaterials
using Printf
using NPZ
using PyPlot
using ProgressMeter

## Charge assignment of Core 2019-FSR

In [2]:
@eval PorousMaterials PATH_TO_CRYSTALS = joinpath(pwd(), "..", "build_graphs", "CoRE_v2_FSR")

"/home/sturlusa/my_projects/scratch/mpn_charges/deployment/../build_graphs/CoRE_v2_FSR"

In [3]:
readdir(PorousMaterials.PATH_TO_CRYSTALS)

6008-element Array{String,1}:
 "ABAVIJ_freeONLY.cif"
 "ABAYIO_freeONLY.cif"
 "ABAYOU_freeONLY.cif"
 "ABEFUL_freeONLY.cif"
 "ABESUX_freeONLY.cif"
 "ABETAE_freeONLY.cif"
 "ABETIN_freeONLY.cif"
 "ABEXEM_freeONLY.cif"
 "ABEXEN_freeONLY.cif"
 "ABEXIQ_freeONLY.cif"
 "ABEXOW_freeONLY.cif"
 "ABEXUC_freeONLY.cif"
 "ABEYAJ_freeONLY.cif"
 ⋮
 "jz4002345_si_006_freeONLY.cif"
 "la304204k_si_003_freeONLY.cif"
 "la4012839_si_002_auto.cif"
 "magnetochemistry3010001_PF290NdNO3_freeONLY.cif"
 "magnetochemistry3010001_PF362NdCl_freeONLY.cif"
 "nchem.2258-s3_freeONLY.cif"
 "ncomms10007_ncomms10007-s3_freeONLY.cif"
 "ncomms11831_ncomms11831-s2_freeONLY.cif"
 "ncomms11831_ncomms11831-s3_freeONLY.cif"
 "ncomms7350-s2_freeONLY.cif"
 "om701262m-file004_freeONLY.cif"
 "science.1056698_manual.cif"

In [4]:
if ! isdir(joinpath(pwd(), "..", "MPNN_CoRE-FSR"))
    mkdir(joinpath(pwd(), "..", "MPNN_CoRE-FSR"))
end
@showprogress for (i_xtal, xtalname) in enumerate(readdir(PorousMaterials.PATH_TO_CRYSTALS))
    if ! isfile(joinpath(pwd(), "results", "predictions", "deployment_graphs_FSR", 
                         split(xtalname, ".cif")[1] * "_gaussian_cor_predictions.npy"))
        continue
    end
    predicted_charges = npzread(joinpath(pwd(), "results", "predictions", 
                                "deployment_graphs_FSR", split(xtalname, ".cif")[1] * "_gaussian_cor_predictions.npy"))
    xtal = Crystal(xtalname)
    new_charges = Charges(xtal.charges.n, Float64.(predicted_charges), xtal.charges.coords)
    new_xtal = Crystal(xtal.name, xtal.box, xtal.atoms, new_charges, xtal.bonds, xtal.symmetry)
    write_cif(new_xtal, joinpath(pwd(), "..", "MPNN_CoRE-FSR", xtalname))
end

Progress: 100%|█████████████████████████████████████████| Time: 0:02:58


## Charge assignment of CoRE 2019-ASR

In [5]:
@eval PorousMaterials PATH_TO_CRYSTALS = joinpath(pwd(), "..", "build_graphs", "CoRE_v2_ASR")

"/home/sturlusa/my_projects/scratch/mpn_charges/deployment/../build_graphs/CoRE_v2_ASR"

In [6]:
readdir(PorousMaterials.PATH_TO_CRYSTALS)

10143-element Array{String,1}:
 "ABAVIJ_clean.cif"
 "ABAYIO_clean.cif"
 "ABAYOU_clean.cif"
 "ABEFUL_clean.cif"
 "ABESUX_clean.cif"
 "ABETAE_clean.cif"
 "ABETIN_clean.cif"
 "ABEXEM_clean.cif"
 "ABEXEN_clean.cif"
 "ABEXIQ_clean.cif"
 "ABEXOW_clean.cif"
 "ABEXUC_clean.cif"
 "ABEYAJ_clean.cif"
 ⋮
 "jz4002345_si_005_clean.cif"
 "jz4002345_si_006_clean.cif"
 "la304204k_si_003_clean.cif"
 "la4012839_si_002_auto.cif"
 "magnetochemistry3010001_PF290NdNO3_clean.cif"
 "magnetochemistry3010001_PF362NdCl_clean.cif"
 "nchem.2258-s3_clean.cif"
 "ncomms11831_ncomms11831-s2_clean.cif"
 "ncomms11831_ncomms11831-s3_clean.cif"
 "ncomms7350-s2_clean.cif"
 "om701262m-file004_clean.cif"
 "science.1056698_manual.cif"

In [7]:
if ! isdir(joinpath(pwd(), "..", "MPNN_CoRE-ASR"))
    mkdir(joinpath(pwd(), "..", "MPNN_CoRE-ASR"))
end
@showprogress for (i_xtal, xtalname) in enumerate(readdir(PorousMaterials.PATH_TO_CRYSTALS))
    if ! isfile(joinpath(pwd(), "results", "predictions", "deployment_graphs_ASR", 
                         split(xtalname, ".cif")[1] * "_gaussian_cor_predictions.npy"))
        continue
    end
    predicted_charges = npzread(joinpath(pwd(), "results", "predictions", "deployment_graphs_ASR", 
                                split(xtalname, ".cif")[1] * "_gaussian_cor_predictions.npy"))
    xtal = Crystal(xtalname)
    new_charges = Charges(xtal.charges.n, Float64.(predicted_charges), xtal.charges.coords)
    new_xtal = Crystal(xtal.name, xtal.box, xtal.atoms, new_charges, xtal.bonds, xtal.symmetry)
    write_cif(new_xtal, joinpath(pwd(), "..", "MPNN_CoRE-ASR", xtalname))
end

Progress:  80%|████████████████████████████████▊        |  ETA: 0:00:47┌ Info: Crystal VEWLAM_clean.cif has P21 space group. I am converting it to P1 symmetry.
│         To afrain from this, pass `convert_to_p1=false` to the `Crystal` constructor.
└ @ PorousMaterials /home/sturlusa/my_projects/PorousMaterials.jl/src/crystal.jl:405
Progress: 100%|█████████████████████████████████████████| Time: 0:03:58
